# A Crash Course on PyTorch

Welcome! This notebook provides a quick introduction to PyTorch, we will cover the entire pipeline from loading data to training a model and saving it.

Finally, we'll explore how to integrate powerful tools like Weights & Biases (wandb) for experiment tracking and Hugging Face for leveraging state-of-the-art pre-trained models.

## Setting Up and Loading the Data

First, let's install the necessary libraries.

In [ ]:
!pip install --upgrade torch torchvision

The Dataset and DataLoader classes encapsulate the process of pulling your data from storage and exposing it to your training loop in batches.

The Dataset is responsible for accessing and processing single instances of data.

The DataLoader pulls instances of data from the Dataset (either automatically or with a sampler that you define), collects them in batches, and returns them for consumption by your training loop. The DataLoader works with all kinds of datasets, regardless of the type of data they contain.

For this tutorial, we’ll be using the Fashion-MNIST dataset provided by TorchVision. We use torchvision.transforms.Normalize() to zero-center and normalize the distribution of the image tile content, and download both training and validation data splits.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from datetime import datetime


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

As always, let’s visualize the data as a sanity check:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)
print('  '.join(classes[labels[j]] for j in range(4)))

## Defining the PyTorch Model

The model we’ll use in this example is a variant of LeNet-5.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# PyTorch models inherit from torch.nn.Module
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = GarmentClassifier()

## Training the Model

With our data and model ready, we can start the training process. This involves:
1.  **Instantiating the model**, loss function, and optimizer.
2.  Creating a **training loop** that iterates over the data for a set number of epochs.
3.  Creating an **evaluation loop** to check the model's performance on the test set.


For this example, we’ll be using a cross-entropy loss. For demonstration purposes, we’ll create batches of dummy output and label values, run them through the loss function, and examine the result.

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

# NB: Loss functions expect data in batches, so we're creating batches of 4
# Represents the model's confidence in each of the 10 classes for a given input
dummy_outputs = torch.rand(4, 10)
# Represents the correct class among the 10 being tested
dummy_labels = torch.tensor([1, 5, 3, 7])

print(dummy_outputs)
print(dummy_labels)

loss = loss_fn(dummy_outputs, dummy_labels)
print('Total loss for this batch: {}'.format(loss.item()))

For this example, we’ll be using simple stochastic gradient descent with momentum.

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### The Training Loop

Below, we have a function that performs one training epoch. It enumerates data from the DataLoader, and on each pass of the loop does the following:

Gets a batch of training data from the DataLoader

Zeros the optimizer’s gradients

Performs an inference - that is, gets predictions from the model for an input batch

Calculates the loss for that set of predictions vs. the labels on the dataset

Calculates the backward gradients over the learning weights

Tells the optimizer to perform one learning step - that is, adjust the model’s learning weights based on the observed gradients for this batch, according to the optimization algorithm we chose

It reports on the loss for every 1000 batches.

Finally, it reports the average per-batch loss for the last 1000 batches, for comparison with a validation run


In [ ]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

### Per-Epoch Activity

There are a couple of things we’ll want to do once per epoch:

    Perform validation by checking our relative loss on a set of data that was not used for training, and report this

    Save a copy of the model


In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

To load a saved version of the model:

In [ ]:
PATH = 'model_20250926_132747_0'
saved_model = GarmentClassifier()
saved_model.load_state_dict(torch.load(PATH))

Once you’ve loaded the model, it’s ready for whatever you need it for - more training, inference, or analysis.

# Integrations: `wandb` and `huggingface`

While the above covers the core PyTorch workflow, real-world projects often use additional tools to boost productivity.

### Using Weights & Biases (`wandb`) for Experiment Tracking

`wandb` is a fantastic tool for logging metrics, visualizing results, and comparing different training runs. It helps you keep track of your experiments effortlessly.

To use it, you'd add a few lines to your training script:
1.  `import wandb`
2.  `wandb.init()` to start a new run.
3.  `wandb.log()` to log metrics like loss and accuracy inside your training loop.

In [ ]:
!pip install wandb -q

import wandb
wandb.login(key="9df9f39261aa2848916f43dd4cb4a4eed854b322")

# 1. Start a new run
wandb.init(project="fashion-mnist-tutorial")

# 2. Log hyperparameters
config = wandb.config
config.learning_rate = 0.001
config.epochs = 3
config.batch_size = 4

# Let's redefine the train and evaluate functions to include wandb.log()
def train_with_wandb(dataloader):
    model.train(True)
    running_loss = 0.
    for i, data in enumerate(dataloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / (i + 1)
    wandb.log({"train_loss": avg_loss})
    return avg_loss

def evaluate_with_wandb(dataloader):
    model.eval()
    running_loss = 0.
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, labels = data
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            running_loss += loss.item()
    avg_loss = running_loss / (i + 1)
    wandb.log({"val_loss": avg_loss})
    return avg_loss 

In [ ]:
print("\n--- Starting Training with WandB Logging ---\n")
# Re-initialize model and optimizer for a fresh run
model = GarmentClassifier()
optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)

for epoch in range(config.epochs):
    print(f'EPOCH {epoch + 1}:')
    train_loss = train_with_wandb(training_loader)
    val_loss = evaluate_with_wandb(validation_loader)
    print(f'  train_loss: {train_loss}  val_loss: {val_loss}\n')

### Using Hugging Face `transformers` for Pre-trained Models

We'll use the datasets library to download and prepare the IMDb dataset. This involves loading the data and then using a tokenizer to convert the text into a format the model can understand.

In [ ]:
!pip install --upgrade transformers datasets evaluate -q

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding

# We load the first 1000 training and 500 testing samples
small_imdb_dataset = DatasetDict({
    'train': load_dataset("imdb", split='train[:1000]'),
    'test': load_dataset("imdb", split='test[:500]')
})

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Apply tokenization to the small dataset
tokenized_imdb = small_imdb_dataset.map(preprocess_function, batched=True)
tokenized_imdb = tokenized_imdb.remove_columns(["text"]).rename_column("label", "labels")

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Next, we'll load the pre-trained DistilBERT model. We use AutoModelForSequenceClassification which automatically adds a classification head on top of the base model. We also define the evaluation metric and the training arguments.

In [ ]:
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Evaluation metric
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

The Trainer API provides a simple way to handle the training loop. We just need to pass it our model, datasets, tokenizer, training configuration, and evaluation function.

In [ ]:
from transformers import TrainingArguments, Trainer

# Define the training arguments
training_args = TrainingArguments(
    output_dir="my_awesome_model",        # Directory to save the model
    learning_rate=2e-5,                   # The learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=2,                   # Number of training epochs
    weight_decay=0.01,                    # Strength of weight decay regularization
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                # Save the model at the end of each epoch
    load_best_model_at_end=True,          # Load the best model found during training
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"], # Training data
    eval_dataset=tokenized_imdb["test"],   # Evaluation data
    tokenizer=tokenizer,
    data_collator=data_collator,           # Our data collator for dynamic padding
    compute_metrics=compute_metrics,       # The function to compute metrics
)

# Start the training process!
trainer.train()